In [1]:
# test_similarity.py

import pandas as pd

def simple_similarity_test(data_path, target_plate, candidate_plate, time_window_minutes=1):
    """
    一個簡化的測試腳本，專門用來驗證兩輛車之間是否存在潛在的接觸點。
    """
    try:
        full_df = pd.read_csv(data_path)
        full_df['datetime'] = pd.to_datetime(full_df['日期'] + ' ' + full_df['時間'])
    except FileNotFoundError:
        print(f"錯誤：找不到資料檔案 {data_path}")
        return

    print(f"--- 正在測試 {target_plate} 與 {candidate_plate} 的相似性 ---")

    target_df = full_df[full_df['車牌'] == target_plate].copy()
    candidate_df = full_df[full_df['車牌'] == candidate_plate].copy()

    # --- 關鍵步驟：尋找「潛在接觸點」 ---
    # 為了高效比對，我們將時間四捨五入到分鐘
    target_df['time_minute'] = target_df['datetime'].dt.round('min')
    candidate_df['time_minute'] = candidate_df['datetime'].dt.round('min')
    
    # 找出在同一個攝影機、且時間差在 ±1 分鐘內的所有紀錄
    # 這是整個演算法的核心
    potential_matches = pd.merge(
        target_df[['攝影機名稱', 'time_minute', 'datetime']],
        candidate_df[['攝影機名稱', 'time_minute', 'datetime']],
        on=['攝影機名稱'],
        suffixes=('_target', '_candidate')
    )
    
    # 計算時間差
    potential_matches['time_diff_minutes'] = (
        (potential_matches['datetime_candidate'] - potential_matches['datetime_target']).dt.total_seconds() / 60
    ).abs()

    # 篩選出真正符合時間窗的接觸點
    contact_points = potential_matches[potential_matches['time_diff_minutes'] <= time_window_minutes]

    # --- 輸出結果 ---
    print(f"\n在 {len(target_df)} 筆目標軌跡和 {len(candidate_df)} 筆候選軌跡中：")
    
    if contact_points.empty:
        print(f"分析結果：找不到任何在 ±{time_window_minutes} 分鐘時間窗內的共同攝影機點。")
        print("這解釋了為什麼相似度分析沒有結果。")
        
        # 為了證明，我們放寬時間窗來看看
        wider_contact_points = potential_matches[potential_matches['time_diff_minutes'] <= 30] # 放寬到 30 分鐘
        if not wider_contact_points.empty:
            print("\n如果我們將時間窗放寬到 ±30 分鐘，可以找到以下接觸點：")
            print(wider_contact_points[['攝影機名稱', 'datetime_target', 'datetime_candidate', 'time_diff_minutes']].head())
        else:
            print("\n即使將時間窗放寬到 ±30 分鐘，也找不到任何接觸點。")

    else:
        print(f"成功找到 {len(contact_points)} 個在 ±{time_window_minutes} 分鐘內的接觸點！")
        print("預覽：")
        print(contact_points[['攝影機名稱', 'datetime_target', 'datetime_candidate', 'time_diff_minutes']].head())


if __name__ == '__main__':
    DATA_FILE_PATH = 'data/vehicle_behavior_dataset_2months_final.csv'
    
    # 測試毒販與探路的跟班
    simple_similarity_test(DATA_FILE_PATH, 'DEL-8888', 'ACC-0001')
    
    print("\n" + "="*50 + "\n")
    
    # 測試毒販與殿後的跟班
    simple_similarity_test(DATA_FILE_PATH, 'DEL-8888', 'ACC-0002')

--- 正在測試 DEL-8888 與 ACC-0001 的相似性 ---

在 275 筆目標軌跡和 100 筆候選軌跡中：
成功找到 24 個在 ±1 分鐘內的接觸點！
預覽：
            攝影機名稱     datetime_target  datetime_candidate  time_diff_minutes
0     平鎮區-C住宅區-巷口 2025-08-02 22:21:02 2025-08-02 22:21:04           0.033333
252   平鎮區-C住宅區-巷口 2025-08-04 23:00:14 2025-08-04 22:59:47           0.450000
277      交通幹道-CP段 2025-08-04 23:08:57 2025-08-04 23:08:47           0.166667
882   平鎮區-C住宅區-巷口 2025-08-16 23:00:01 2025-08-16 22:59:17           0.733333
1008  平鎮區-C住宅區-巷口 2025-08-17 22:52:10 2025-08-17 22:52:11           0.016667


--- 正在測試 DEL-8888 與 ACC-0002 的相似性 ---

在 275 筆目標軌跡和 100 筆候選軌跡中：
成功找到 23 個在 ±1 分鐘內的接觸點！
預覽：
           攝影機名稱     datetime_target  datetime_candidate  time_diff_minutes
0    平鎮區-C住宅區-巷口 2025-08-02 22:21:02 2025-08-02 22:21:59           0.950000
126  平鎮區-C住宅區-巷口 2025-08-03 22:30:24 2025-08-03 22:29:53           0.516667
201  大溪區-P停車場-後方 2025-08-03 23:00:35 2025-08-03 23:01:14           0.650000
252  平鎮區-C住宅區-巷口 2025-08-04 23:00:14 2025-08-04 23

In [3]:
import pandas as pd

# --- 1. 設定檔案路徑 ---
file_path = r'C:\Users\user\Desktop\宇統資訊\LLM_Report_Service_v1\data\vehicle_behavior_dataset_2months_final.csv'

# --- 2. 設定您想顯示的欄位 ---
columns_to_show = ['攝影機名稱', '車牌',  '日期', '時間']

try:
    # --- 3. 讀取 CSV 檔案 ---
    df = pd.read_csv(file_path, encoding='utf-8')

    # --- 4. 篩選資料 ---
    # 找到 '車牌' 為 'NOR-1001' 的資料
    filtered_df = df[df['車牌'] == 'NOR-1001']

    # --- 5. 選取指定的欄位並取得前 20 筆 ---
    # 在篩選後的結果中，只選取 columns_to_show 中指定的欄位
    result = filtered_df[columns_to_show].head(20)

    # --- 6. 印出結果 ---
    if result.empty:
        print("資料集中找不到車牌為 'NOR-1001' 的紀錄。")
    else:
        print("成功找到車牌為 'NOR-1001' 的前 20 筆資料 (指定欄位)：")
        # 直接印出 DataFrame，pandas 會自動對齊，非常整齊
        print(result.to_string(index=False))


except FileNotFoundError:
    print(f"錯誤：找不到指定的檔案。請再次確認路徑是否正確：\n{file_path}")
except KeyError as e:
    print(f"錯誤：找不到指定的欄位名稱 {e}。請檢查 CSV 檔案中的欄位名稱是否正確。")
except Exception as e:
    print(f"讀取或處理檔案時發生錯誤：{e}")

成功找到車牌為 'NOR-1001' 的前 20 筆資料 (指定欄位)：
      攝影機名稱       車牌         日期       時間
楊梅區-A住宅區-巷口 NOR-1001 2025-08-01 08:32:38
   楊梅區-交流道A NOR-1001 2025-08-01 08:41:15
   交通幹道-AB段 NOR-1001 2025-08-01 08:49:46
   中壢區-交流道B NOR-1001 2025-08-01 09:00:13
 中壢區-B園區-入口 NOR-1001 2025-08-01 09:12:02
 中壢區-B園區-內部 NOR-1001 2025-08-01 09:21:19
 中壢區-B園區-內部 NOR-1001 2025-08-01 09:21:19
 中壢區-B園區-內部 NOR-1001 2025-08-01 17:37:15
 中壢區-B園區-內部 NOR-1001 2025-08-01 18:07:06
 中壢區-B園區-入口 NOR-1001 2025-08-01 18:16:16
   中壢區-交流道B NOR-1001 2025-08-01 18:25:04
   交通幹道-AB段 NOR-1001 2025-08-01 18:35:50
   楊梅區-交流道A NOR-1001 2025-08-01 18:47:28
楊梅區-A住宅區-巷口 NOR-1001 2025-08-01 18:56:17
楊梅區-A住宅區-巷口 NOR-1001 2025-08-04 08:16:28
   楊梅區-交流道A NOR-1001 2025-08-04 08:26:48
   交通幹道-AB段 NOR-1001 2025-08-04 08:37:05
   中壢區-交流道B NOR-1001 2025-08-04 08:46:35
 中壢區-B園區-入口 NOR-1001 2025-08-04 08:56:32
 中壢區-B園區-內部 NOR-1001 2025-08-04 09:06:16
